In [237]:
from IPython.display import display
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

def imshow(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(image)
    plt.figure(figsize=(10,10))
    plt.imshow(pil_img)
    plt.axis('off')
    plt.show()

def preprocess(image):
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

In [238]:
# clicked_points = []
_last_H = None

# def _on_mouse(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         clicked_points.append((x, y))
#         msg = f"click: ({x}, {y})"
#         # Also print board-space if homography is available
#         try:
#             if _last_H is not None:
#                 H_inv = np.linalg.inv(_last_H)  # camera -> board
#                 p = np.array([[[x, y]]], dtype=np.float32)
#                 q = cv2.perspectiveTransform(p, H_inv)[0, 0]
#                 msg += f" | board: ({int(q[0])}, {int(q[1])})"
#         except np.linalg.LinAlgError:
#             pass
#         print(msg)

def process_video(file, show=True):
    in_path = f"data/{file}.mp4"
    cap = cv2.VideoCapture(in_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = fps if fps and fps > 0 else 25.0
    delay_ms = int(1000 / fps)

    paused = False
    frame_out = None

    if show:
        cv2.namedWindow("video")
        # cv2.setMouseCallback("video", _on_mouse)

    while True:
        if not paused or frame_out is None:
            ret, frame = cap.read()
            if not ret:
                break
            frame_out = pipeline(frame)

        # # draw clicked points on the shown frame
        # for pt in clicked_points:
        #     if 0 <= pt[0] < frame_out.shape[1] and 0 <= pt[1] < frame_out.shape[0]:
        #         cv2.circle(frame_out, pt, 5, (0, 255, 0), -1)

        if show:
            cv2.imshow("video", frame_out)

        key = cv2.waitKey(0 if paused else delay_ms) & 0xFF
        if key == ord("q"):
            break
        elif key == ord(" "):  # spacebar
            paused = not paused

    cap.release()
    cv2.destroyAllWindows()

In [239]:
import subprocess
import os

def process_video(file, show=True, duration_sec=30):
    in_path = f"data/{file}.mp4"
    temp_path = f"temp_{file}.avi"
    out_path = f"data/{file}_output.mp4"
    
    cap = cv2.VideoCapture(in_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {in_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps = fps if fps and fps > 0 else 25.0
    delay_ms = int(1000 / fps)
    
    total_frames = int(fps * duration_sec)
    frame_count = 0

    # Use MJPEG codec (more reliable than mp4v)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(temp_path, fourcc, fps, (frame_width, frame_height))

    if not out.isOpened():
        print("Error: Cannot open VideoWriter")
        cap.release()
        return

    paused = False
    frame_out = None

    if show:
        cv2.namedWindow("video")

    print(f"Processing {total_frames} frames...")
    
    while frame_count < total_frames:
        if not paused or frame_out is None:
            ret, frame = cap.read()
            if not ret:
                break
            frame_out = pipeline(frame)
            out.write(frame_out)
            frame_count += 1

        if show:
            cv2.imshow("video", frame_out)

        key = cv2.waitKey(0 if paused else delay_ms) & 0xFF
        if key == ord("q"):
            break
        elif key == ord(" "):
            paused = not paused

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"✓ Temp video saved: {temp_path}")
    
    # Convert to MP4 with ffmpeg
    print(f"Converting to {out_path}...")
    cmd = [
        "ffmpeg", "-i", temp_path, 
        "-c:v", "libx264", "-crf", "23",
        "-y", out_path
    ]
    
    try:
        subprocess.run(cmd, check=True, capture_output=True)
        print(f"✓ Video saved: {out_path}")
        os.remove(temp_path)
    except subprocess.CalledProcessError as e:
        print(f"Error: ffmpeg failed - {e.stderr.decode()}")

In [240]:
def get_mask_of_board(frame_bgr):
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, np.array([20, 40, 40]), np.array([140, 255, 255]))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    filtered = [c for c in contours if cv2.contourArea(c) > 500]
    
    all_points = np.vstack(filtered)
    hull = cv2.convexHull(all_points)
    
    board_mask = np.zeros_like(mask)
    cv2.drawContours(board_mask, [hull], -1, 255, -1)
    
    return board_mask

BOARD_IMG = cv2.imread("data/board.jpg")
BOARD_IMG_MASK = get_mask_of_board(BOARD_IMG)

In [241]:
def preprocess_for_features(bgr):
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    return gray

def find_H(template_bgr, frame_bgr, template_mask=None):
    # ORB works best on single-channel images
    img1 = preprocess_for_features(template_bgr)
    img2 = preprocess_for_features(frame_bgr)

    orb = cv2.ORB_create(
        nfeatures=6000,
        scaleFactor=1.2,
        nlevels=8,
        fastThreshold=10,
    )

    k1, d1 = orb.detectAndCompute(img1, template_mask)
    k2, d2 = orb.detectAndCompute(img2, None)

    if d1 is None or d2 is None or len(k1) < 8 or len(k2) < 8:
        return None, 0

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    matches = bf.knnMatch(d1, d2, k=2)

    good = []
    for pair in matches:
        if len(pair) != 2:
            continue
        m, n = pair
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) < 25:
        return None, 0

    src = np.float32([k1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst = np.float32([k2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # Prefer USAC if available (OpenCV builds vary)
    method = cv2.RANSAC
    if hasattr(cv2, "USAC_MAGSAC"):
        method = cv2.USAC_MAGSAC

    H, inlier_mask = cv2.findHomography(src, dst, method, 3.0)

    if H is None or inlier_mask is None:
        return None, 0

    inliers = int(inlier_mask.ravel().sum())
    
    return H, inliers


_last_H = None

In [242]:
def draw_board(frame, H_use):
    points = [(480,590), (3000,570), (2987,3141), (445,3088)]
    points_cam = []
    center = (1707,1849)
    output = frame.copy()
    for pt in points:
        board_point = np.array([[[pt[0], pt[1]]]], dtype=np.float32)
        camera_point = cv2.perspectiveTransform(board_point, H_use)
        pt_cam = tuple(map(int, camera_point[0][0]))
        points_cam.append(pt_cam)
    #draw a box around the board
    # put text "BOARD" above the box
    # cv2.putText(output, "BOARD", (points_cam[3][0], points_cam[3][1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    cv2.line(output, points_cam[0], points_cam[1], (255, 0, 0), 5)
    cv2.line(output, points_cam[1], points_cam[2], (255, 0, 0), 5)
    cv2.line(output, points_cam[2], points_cam[3], (255, 0, 0), 5)
    cv2.line(output, points_cam[3], points_cam[0], (255, 0, 0), 5)

    board_center = np.array([[[center[0], center[1]]]], dtype=np.float32)
    camera_center = cv2.perspectiveTransform(board_center, H_use)
    center_cam = tuple(map(int, camera_center[0][0]))
    # cv2.circle(output, center_cam, 7, (0, 0, 255), -1)
    return output

In [243]:
board_points = [(1529, 2852), (1524, 2691), (1529, 2517), (1529, 2348), (1538, 2188), (1364, 2179), (1364, 2005), (1194, 2005), (1030, 2009), (856, 2014), (700, 2009), (530, 2001), (535, 1831), (713, 1657), (865, 1662), (1038, 1666), (1199, 1671), (1359, 1662), (1359, 1497), (1542, 1506), (1542, 1332), (1551, 1163), (1542, 1002), (1542, 833), (1556, 664), (1730, 655), (1872, 1501), (1872, 1341), (1881, 1163), (1881, 993), (2037, 1506), (2051, 1671), (2211, 1684), (2394, 1693), (2563, 1693), (2737, 1689), (2907, 1689), (2907, 1689), (2911, 1858), (2911, 2032), (2711, 2032), (2550, 2027), (2394, 2023), (2211, 2023), (2033, 2023), (2037, 2188), (1868, 2183), (1863, 2353), (1868, 2535), (1859, 2682), (1859, 2870), (1863, 3034), (1694, 3052)]


In [244]:
def draw_pawns(frame, es, H_use):
    # NOTE: 'es' is no longer used (kept in signature so your pipeline doesn't break)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- Hue-based masks (OpenCV H is [0..179]) ---
    # Tune these if needed.
    sat_min, val_min = 150, 50

    # Red wraps around 0 => two ranges
    red1 = cv2.inRange(hsv, (0,   sat_min, val_min), (10,  255,    255))
    red2 = cv2.inRange(hsv, (170, sat_min, val_min), (179, 255,    255))
    red_mask = cv2.bitwise_or(red1, red2)

    yellow_mask = cv2.inRange(hsv, (18, 160, 130), (38, 255, 255))
    green_mask  = cv2.inRange(hsv, (40, 80, val_min), (85, 255, 255))
    blue_mask   = cv2.inRange(hsv, (90, sat_min, val_min), (135, 255, 255))

    # Clean masks
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    def clean(m):
        m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kernel, iterations=1)
        m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=2)
        return m

    red_mask = clean(red_mask)
    yellow_mask = clean(yellow_mask)
    green_mask = clean(green_mask)
    blue_mask = clean(blue_mask)

    # Points are in BOARD space, project to CAMERA and check mask presence there
    r = 6  # radius of the local check AND the drawn dot size
    board_points = [
        (1529, 2852), (1524, 2691), (1529, 2517), (1529, 2348), (1538, 2188),
        (1364, 2179), (1364, 2005), (1194, 2005), (1030, 2009), (856, 2014),
        (700, 2009), (530, 2001), (535, 1831), (713, 1657), (865, 1662),
        (1038, 1666), (1199, 1671), (1359, 1662), (1359, 1497), (1542, 1506),
        (1542, 1332), (1551, 1163), (1542, 1002), (1542, 833), (1556, 664),
        (1730, 655), (1872, 1501), (1872, 1341), (1881, 1163), (1881, 993),
        (2037, 1506), (2051, 1671), (2211, 1684), (2394, 1693), (2563, 1693),
        (2737, 1689), (2907, 1689), (2911, 1858), (2911, 2032), (2711, 2032),
        (2550, 2027), (2394, 2023), (2211, 2023), (2033, 2023), (2037, 2188),
        (1868, 2183), (1863, 2353), (1868, 2535), (1859, 2682), (1859, 2870),
        (1863, 3034), (1694, 3052),
    ]

    h, w = frame.shape[:2]

    for pt in board_points:
        board_point = np.array([[[pt[0], pt[1]]]], dtype=np.float32)
        cam_point = cv2.perspectiveTransform(board_point, H_use)
        x, y = map(int, cam_point[0, 0])

        if not (0 <= x < w and 0 <= y < h):
            continue

        # local circular ROI mask
        circle_mask = np.zeros((h, w), dtype=np.uint8)
        cv2.circle(circle_mask, (x, y), r, 255, -1)

        # Decide color by overlap (priority order: red, green, yellow, blue)
        if np.any(cv2.bitwise_and(red_mask, circle_mask)):
            cv2.circle(frame, (x, y), 7, (0, 0, 255), -1)       # BGR red
        elif np.any(cv2.bitwise_and(green_mask, circle_mask)):
            cv2.circle(frame, (x, y), 7, (0, 255, 0), -1)       # BGR green
        elif np.any(cv2.bitwise_and(yellow_mask, circle_mask)):
            cv2.circle(frame, (x, y), 7, (0, 255, 255), -1)     # BGR yellow
        elif np.any(cv2.bitwise_and(blue_mask, circle_mask)):
            cv2.circle(frame, (x, y), 7, (255, 0, 0), -1)       # BGR blue

    return frame

In [245]:
import cv2
import numpy as np

# --- 1. CORE DETECTION LOGIC (Helper) ---

def get_cup_confidence(contour, frame, hsv_frame):
    """ 
    Scores a contour to see if it behaves like a black cup.
    Returns: score (0-100), bounding_box
    """
    area = cv2.contourArea(contour)
    # Filter 1: Area (Cups are significantly larger than dice dots)
    if area < 5000 or area > 30000: 
        return 0, None
    
    x, y, w, h = cv2.boundingRect(contour)
    
    # Filter 2: Aspect Ratio
    aspect_ratio = float(w) / h
    if aspect_ratio < 0.5 or aspect_ratio > 1.5:
        return 0, None

    # Filter 3: Solidity (The "Blob-ness")
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    if hull_area == 0: return 0, None
    solidity = float(area) / hull_area
    
    if solidity < 0.85:
        return 0, None

    # --- SCORING ---
    
    # Score A: Darkness
    mask = np.zeros(hsv_frame.shape[:2], dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    
    mean_val = cv2.mean(hsv_frame[:,:,2], mask=mask)[0]
    
    # If mean_val is > 60, it's probably gray, not black.
    darkness_score = max(0, min(100, (60 - mean_val) * 2.5))
    
    # Score B: Shape Idealism
    ratio_diff = abs(1.0 - aspect_ratio)
    shape_score = max(0, 100 - (ratio_diff * 100))
    
    # Final Weighted Score
    final_score = (darkness_score * 0.7) + (shape_score * 0.3)
    
    return final_score, (x, y, x + w, y + h)

# --- 2. MAIN FUNCTION (Returns Data) ---

def detect_cup_instant(frame):
    """
    Returns a dictionary with cup data or None if no cup is found.
    Format: {'bbox': (x1, y1, x2, y2), 'score': float, 'contour': np.array}
    """
    # 1. Preprocessing
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    blurred_hsv = cv2.GaussianBlur(hsv, (9, 9), 0)

    # 2. Segmentation (Finding Black)
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 60]) 
    
    mask = cv2.inRange(blurred_hsv, lower_black, upper_black)
    
    # Clean up the mask
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # 3. Find Contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    best_candidate = None
    max_score = 0

    # 4. Evaluate Candidates
    for cnt in contours:
        score, bbox = get_cup_confidence(cnt, frame, hsv)
        
        if score > 50:
            if score > max_score:
                max_score = score
                best_candidate = {
                    'bbox': bbox,     # Tuple (x1, y1, x2, y2)
                    'score': score,   # Float 0-100
                    'contour': cnt    # The raw contour points
                }

    return best_candidate

In [246]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist

# --- HELPER FUNCTIONS (Kept mostly the same) ---

def detect_white_dots(frame):
    """Detect potential dots (candidates)"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    _, binary = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    dots = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 5 < area < 1200: 
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0: continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            if circularity > 0.25:
                M = cv2.moments(cnt)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    dots.append({
                        'center': (cx, cy),
                        'contour': cnt,
                        'area': area,
                        'radius': int(np.sqrt(area/np.pi)) 
                    })
    return dots

def is_dot_on_black_surface(frame, dot):
    """Checks if the immediate ring around a dot is dark."""
    cx, cy = dot['center']
    r = dot['radius']
    
    inner_r = r + 2
    outer_r = r + 8
    
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.circle(mask, (cx, cy), outer_r, 255, -1)
    cv2.circle(mask, (cx, cy), inner_r, 0, -1)
    
    surround_pixels = frame[mask == 255]
    if surround_pixels.size == 0: return False
    
    if len(surround_pixels.shape) == 3: 
        surround_gray = cv2.cvtColor(surround_pixels.reshape(-1, 1, 3), cv2.COLOR_BGR2GRAY)
        avg_brightness = np.mean(surround_gray)
    else:
        avg_brightness = np.mean(surround_pixels)

    if avg_brightness > 100:
        return False 
        
    return True

def get_cluster_score(frame, dot_group):
    """Calculates a score for the potential dice cluster."""
    centers = [d['center'] for d in dot_group]
    x_coords = [c[0] for c in centers]
    y_coords = [c[1] for c in centers]
    
    padding = 15
    x1 = max(0, min(x_coords) - padding)
    y1 = max(0, min(y_coords) - padding)
    x2 = min(frame.shape[1], max(x_coords) + padding)
    y2 = min(frame.shape[0], max(y_coords) + padding)
    
    roi = frame[y1:y2, x1:x2]
    if roi.size == 0: return -100, (0,0,0,0)

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    avg_brightness = np.mean(hsv[:,:,2])
    
    score = 100
    if avg_brightness > 110: score -= (avg_brightness * 3)
    else: score += (150 - avg_brightness)
        
    return score, (x1, y1, x2-x1, y2-y1)

# --- MAIN FUNCTION (Updated) ---

def detect_dice_strict(frame):
    """
    Returns a dictionary with the dice info, or None if no dice is found.
    Format: { 'number': int, 'bbox': (x, y, w, h), 'dots': [list of dot centers] }
    """
    height, width = frame.shape[:2]
    
    all_dots = detect_white_dots(frame)
    if not all_dots: return None

    # 1. Cluster to find the Dice Box
    search_radius = height // 8
    centers = np.array([d['center'] for d in all_dots])
    dist_matrix = cdist(centers, centers)
    visited = [False] * len(all_dots)
    candidates = []
    
    for i in range(len(all_dots)):
        if visited[i]: continue
        cluster = [all_dots[i]]
        visited[i] = True
        queue = [i]
        while queue:
            curr = queue.pop(0)
            neighbors = np.where((dist_matrix[curr] < search_radius) & (visited == False))[0]
            for n in neighbors:
                visited[n] = True
                cluster.append(all_dots[n])
                queue.append(n)
        
        if len(cluster) > 0:
            score, bbox = get_cluster_score(frame, cluster)
            if score > 0:
                candidates.append({'score': score, 'bbox': bbox})

    # 2. Process the Best Candidate
    if candidates:
        candidates.sort(key=lambda x: x['score'], reverse=True)
        best = candidates[0]
        x, y, w, h = best['bbox']
        
        # Expand box for second pass
        expansion = 20
        bx1 = max(0, x - expansion)
        by1 = max(0, y - expansion)
        bx2 = min(width, x + w + expansion)
        by2 = min(height, y + h + expansion)
        
        valid_dots = []
        for dot in all_dots:
            cx, cy = dot['center']
            
            # Check A: Geometrically inside
            if bx1 < cx < bx2 and by1 < cy < by2:
                # Check B: On black surface
                if is_dot_on_black_surface(frame, dot):
                    valid_dots.append(dot['center'])
        
        return {
            'number': len(valid_dots),
            'bbox': (x, y, w, h),
            'dots': valid_dots # Optional: return exact dot locations
        }

    return None

In [247]:
# --- Cup tracking (Kalman) ---

class CupKalmanTracker:
    """
    Tracks cup center (x,y) with constant-velocity model:
      state = [x, y, vx, vy]
      meas  = [x, y]
    """
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)

        self.kf.transitionMatrix = np.array(
            [[1, 0, 1, 0],
             [0, 1, 0, 1],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], dtype=np.float32
        )
        self.kf.measurementMatrix = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0]], dtype=np.float32
        )

        self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
        self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 5e-1
        self.kf.errorCovPost = np.eye(4, dtype=np.float32)

        self.initialized = False
        self.last_wh = None

    def reset(self, x, y, wh=None):
        self.kf.statePost = np.array([[x], [y], [0], [0]], dtype=np.float32)
        self.initialized = True
        if wh is not None:
            self.last_wh = wh

    def update_from_bbox(self, bbox):
        x1, y1, x2, y2 = bbox
        cx = 0.5 * (x1 + x2)
        cy = 0.5 * (y1 + y2)
        w = max(1, x2 - x1)
        h = max(1, y2 - y1)

        if not self.initialized:
            self.reset(cx, cy, wh=(w, h))
            return (cx, cy)

        self.kf.predict()
        meas = np.array([[cx], [cy]], dtype=np.float32)
        est = self.kf.correct(meas)
        self.last_wh = (w, h)
        return (float(est[0, 0]), float(est[1, 0]))

    def predict_only(self):
        if not self.initialized:
            return None
        pred = self.kf.predict()
        return (float(pred[0, 0]), float(pred[1, 0]))

_cup_tracker = CupKalmanTracker()

In [ ]:
_last_dice_number = None
_dice_thrown_ttl = 0

def pipeline(frame):
    global _last_H
    global _last_dice_number, _dice_thrown_ttl
    
    H, inliers = find_H(BOARD_IMG, frame, template_mask=BOARD_IMG_MASK)

    if H is not None and inliers >= 30:
        _last_H = H
    elif _last_H is None:
        return frame

    H_use = _last_H

    try:
        H_inv = np.linalg.inv(H_use)
    except np.linalg.LinAlgError:
        print("Warning: Singular homography matrix, skipping frame")
        return frame

    out_w, out_h = BOARD_IMG.shape[1], BOARD_IMG.shape[0]
    rectified = cv2.warpPerspective(frame, H_inv, (out_w, out_h))

    frame = draw_pawns(frame, rectified, H_use)

    dice_data = detect_dice_strict(frame)
    if dice_data:
        x, y, w, h = dice_data['bbox']
        number = dice_data['number']
        dots = dice_data['dots']

        if _last_dice_number is not None and number != _last_dice_number:
            _dice_thrown_ttl = 20
        _last_dice_number = number

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)

        label = f"Dice: {number}"
        cv2.putText(frame, label, (x, y-15),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

        for center in dots:
            cv2.circle(frame, center, 6, (0, 0, 255), -1)

    if _dice_thrown_ttl > 0:
        cv2.putText(
            frame, "dice was thrown",
            (30, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3
        )
        _dice_thrown_ttl -= 1

    best_candidate = detect_cup_instant(frame)
    if best_candidate:
        x1, y1, x2, y2 = best_candidate["bbox"]
        score = best_candidate["score"]
        contour = best_candidate["contour"]

        # Update tracker using detected bbox center
        cx, cy = _cup_tracker.update_from_bbox((x1, y1, x2, y2))

        # Draw detection (as you already do)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 3)
        cv2.putText(
            frame, f"Cup: {int(score)}%",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2
        )
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)

        # Optional: draw tracked center
        cv2.circle(frame, (int(cx), int(cy)), 5, (255, 255, 0), -1)

    else:
        # No detection this frame -> just predict and draw a tracked box
        pred = _cup_tracker.predict_only()
        if pred is not None and _cup_tracker.last_wh is not None:
            cx, cy = pred
            bw, bh = _cup_tracker.last_wh
            x1 = int(cx - bw / 2)
            y1 = int(cy - bh / 2)
            x2 = int(cx + bw / 2)
            y2 = int(cy + bh / 2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(
                frame, "Cup: track",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2
            )
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 3)
        
        # Label
        label = f"Cup"
        cv2.putText(frame, label, (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        
        # Draw Contour overlay
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
    frame = draw_board(frame, H_use)

 
    return frame
process_video("pro1", show=True, duration_sec=10)

Processing 300 frames...
✓ Temp video saved: temp_pro1.avi
Converting to data/pro1_output.mp4...
